In [66]:
import psycopg2
import boto3
import os
import pandas as pd
import pickle

from dotenv import load_dotenv

from rds_connector import rds_connection
from rds_connector import list_tables
from rds_connector import sql_query
from rds_connector import rds_sql_pull

from teams_notifications import channel_notification

# Load environment variables from .env file
load_dotenv()

# Access the secret value
username = os.getenv('USER')
password = os.getenv('PASS')
db = os.getenv('DB')
server = os.getenv('SERVER')

try:

    #Connect to AWS RDS Database
    conn, cursor = rds_connection(username,
                          password,
                          db,
                          server)
    
except Exception as e:
    
    #Raise Error
    raise(f"AWS RDS CONNECTION ERROR: {e}")

In [95]:
class RDSTablePull:

    def __init__(self, conn, cursor, query, schema):
        self.conn = conn
        self.cursor = cursor
        self.query = query
        self.schema = schema
        self.df = pd.DataFrame()
        self.fields_missing = []
        

    def check_schema(self, df):

        #Set Check
        check = True
        
        #Check if DF Empty
        if df.empty == False:
        
            #Check if Fields Exist in DF
            for field in self.schema:
                if field not in df.columns:
                    
                    #Field not Found, Set Check to False and Add Missing Field to List
                    check = False
                    self.fields_missing.append(field)

        else:
            raise Exception("Error: Dataframe Empty When Checking Schema")

        return check
    

    
    def update_field_names(self, table_fields, new_fields):
        
        #Create Dictionary for Fields Update
        field_dict = dict(zip(table_fields, new_fields))

        #Update Fields
        for table_field, new_field in field_dict.items():

            if self.df.empty == False :
            
                try:
                    #Update the Column Name
                    self.df.rename(columns={table_field: new_field}, inplace=True)

                except Exception as e:
                    raise Exception(f"Failure to Convert {table_field} to {new_field}")
    
            else:
                raise Exception("Error: Cannot Rename Columns of Empty Dataframe")



    def query_to_df(self, update_col = False, table_fields = None, new_fields = None):
        
        #Update DataFrame with SQL Query
        data = rds_sql_pull(self.cursor, self.query)

        #Check if Data Empty
        if data.empty == False:
            #Check Schema
            if self.check_schema(data):
                
                #Update DF with Data
                self.df = data

                #If Selected, Update Columns
                if update_col == True:
                    self.update_field_names(table_fields, new_fields)

                #Return DataFrame
                return self.df
            
            else:
                print(f'Missing Fields from Table:  {self.fields_missing}')
                raise Exception("Error: DataFrame Schema Did Not Match, Check fields_missing()")
            
        else:
            raise Exception("Error: DataFrame Emtpy from SQL Query")

        

    


In [73]:
# Define your SQL query
query = '''
SELECT 
    apps.application_number, 
    apps.created_at, 
    hotel_name.value AS HotelName,
    hotel_address.line1 AS HotelAddressLine1,
    hotel_address.line2 AS HotelAddressLine2,
    hotel_address.city AS HotelCity,
    hotel_address.state AS HotelState,
    hotel_address.zip AS HotelZip,
    hotel_status.value AS HotelStatus
FROM 
    applications_application apps
LEFT JOIN 
    application_data_answer data_answers_1 ON apps.id = data_answers_1.application_id AND data_answers_1.question_id = 1015
LEFT JOIN 
    application_data_textboxanswer hotel_name ON data_answers_1.id = hotel_name.answer_ptr_id
LEFT JOIN 
    application_data_answer data_answers_2 ON data_answers_1.repeating_answer_section_id = data_answers_2.repeating_answer_section_id AND data_answers_2.question_id = 1016
LEFT JOIN 
    application_data_addressanswer hotel_address ON data_answers_2.id = hotel_address.answer_ptr_id
LEFT JOIN 
    application_data_answer data_answers_3 ON data_answers_1.repeating_answer_section_id = data_answers_3.repeating_answer_section_id AND data_answers_3.question_id = 1013
LEFT JOIN 
    application_data_singleselectanswer hotel_status ON data_answers_3.id = hotel_status.answer_ptr_id
WHERE 
    apps.project_id = 34
ORDER BY 
    apps.id;
'''

In [91]:
schema = [
      'hotelname',
      'hoteladdressline1',
      'hoteladdressline2',
      'hotelcity',
      'hotelstate',
      'hotelzip'
]

new_fields = [
    'Hotel_Name',
    'Hotel_Address_Line1',
    'Hotel_Address_Line2',
    'Hotel_City',
    'Hotel_State',
    'Hotel_Zip'
]

In [96]:
rds = RDSTablePull(conn,
                   cursor,
                   query,
                   schema)

In [97]:
rds.query_to_df(update_col = True, 
                table_fields = schema, 
                new_fields = new_fields)


,application_number,created_at,Hotel_Name,Hotel_Address_Line1,Hotel_Address_Line2,Hotel_City,Hotel_State,Hotel_Zip,hotelstatus
0,KQ6625DD77,2024-09-27 18:40:42.775742+00:00,6846846489,,,,FL,,Applicant Accepted
1,KQ6625DD77,2024-09-27 18:40:42.775742+00:00,Testing 1234,dfsijldfsaijpldfs,,,FL,,Rejected
2,B2B9MZYTQL,2024-10-01 18:06:55.607877+00:00,,,,,FL,,Rejected
3,NY6LC7XV4W,2024-10-01 19:10:39.192189+00:00,None,None,None,None,None,None,None
4,4VD5687DFG,2024-10-01 20:01:35.440307+00:00,None,None,None,None,None,None,None
5,2Y9YFK89FM,2024-10-01 20:43:52.325004+00:00,None,None,None,None,None,None,None
6,T8VE6G67MB,2024-10-01 20:54:00.524223+00:00,,,,,FL,,Pending Booking
7,PTJ6LZ66GL,2024-10-01 20:59:15.228480+00:00,None,None,None,None,None,None,None
8,MUPLN8576P,2024-10-01 21:16:58.612302+00:00,None,None,None,None,None,None,None
9,VPEJ5TV8AB,2024-10-01 21:17:57.148812+00:00,None,None,None,None,None,None,None


In [90]:
rds.fields_missing

['bbbek']

In [ ]:


# AWS credentials
#ACCESS_KEY = os.getenv("ACESSKEY")
#SECRET_KEY = os.getenv("SECRETKEY")

# Bucket and file information
bucket_name = 'your_bucket_name'
file_path = 'path_to_your_csv_file.csv'
key_name = 'key_name_for_csv_file_in_s3_bucket.csv'

# Create a S3 client
s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)

# Upload the CSV file to S3 bucket
s3.upload_file(file_path, bucket_name, key_name)

print(f'{file_path} has been successfully uploaded to {bucket_name}/{key_name}')

In [69]:
class RDSTable:

    def __init__(self, queries):
        self.queries = queries


    def data_tables(self):
        
        #Create Empty Table Dictionary
        data_tables = {}

        #Cycle Through Queries and Create Data Tables
        for query in self.queries:
            
            df = rds_sql_pull(cursor, query['query'])

            query_dict = query
            query_dict['df'] = df

            data_tables[query['name']] = query_dict

        print(data_tables)
        
        return data_tables






    def compare_columns(self):
        pass

    def connect_table(self):
        pass